In [ ]:
SRC_LANG = "eng_Latn"
TGT_LANG = "hau_Latn"

src_key = "sentence_" + SRC_LANG
tgt_key = "sentence_" + TGT_LANG

In [ ]:
path_TMX_data_eng = "Kanuri_TMs_EN.txt"
path_TMX_data_knc = "Kanuri_TMs_KR.txt"

data_TMX = []
data_TMX_other = []

size_devtest = 1000

with open(path_TMX_data_eng) as f_en, open(path_TMX_data_knc) as f_kn:
    for l_en, l_kn in zip(f_en.readlines(), f_kn.readlines()):
        sent_en = l_en.strip()
        sent_kn = l_kn.strip()
        
        if len(sent_en) > 2:
            data_TMX.append({src_key:sent_en, tgt_key:sent_kn})
        else:
            data_TMX_other.append({src_key:sent_en, tgt_key:sent_kn})

In [ ]:
data_TMX[0:5]

In [ ]:
from random import shuffle

In [ ]:
shuffle(data_TMX)

In [ ]:
data_TMX_dev = data_TMX[0:1000]
data_TMX_test = data_TMX[1000:2000]
data_TMX_train = data_TMX[2000:] + data_TMX_other
shuffle(data_TMX_train)
print("data_TMX_train", len(data_TMX_train))
print("data_TMX_dev", len(data_TMX_dev))
print("data_TMX_test", len(data_TMX_test))

In [ ]:
data_TMX_dev[0:5]

# Load FLORES dataset

In [ ]:
path_FLORES_data_eng_dev = "FLORES/eng_Latn.dev"
path_FLORES_data_knc_dev = "FLORES/knc_Latn.dev"

path_FLORES_data_eng_test = "FLORES/eng_Latn.devtest"
path_FLORES_data_knc_test = "FLORES/hau_Latn.devtest"

In [ ]:
data_FLORES_test = []
data_FLORES_dev = []

with open(path_FLORES_data_eng_test) as f_en, open(path_FLORES_data_knc_test) as f_kn:
    for l_en, l_kn in zip(f_en.readlines(), f_kn.readlines()):
        data_FLORES_test.append({src_key:l_en.strip(), tgt_key:l_kn.strip()})
        
# with open(path_FLORES_data_eng_dev) as f_en, open(path_FLORES_data_knc_dev) as f_kn:
#     for l_en, l_kn in zip(f_en.readlines(), f_kn.readlines()):
#         data_FLORES_dev.append({src_key:l_en.strip(), tgt_key:l_kn.strip()})

print("data_FLORES_test", len(data_FLORES_test))
# print("data_FLORES_dev", len(data_FLORES_dev))

In [ ]:
# Store data as JSON files
import json

In [ ]:
# with open('fine-tuning-data/FLORES_dev.json', 'w',encoding="utf-8") as file:
#     json.dump(data_FLORES_dev, file, indent=4)
    
with open('FLORES_test_hau-eng.json', 'w',encoding="utf-8") as file:
    json.dump(data_FLORES_test, file, indent=4)

In [ ]:
with open('fine-tuning-data/TMX_test.json', 'w',encoding="utf-8") as file:
    json.dump(data_TMX_test, file, indent=4)
    
with open('fine-tuning-data/TMX_train.json', 'w',encoding="utf-8") as file:
    json.dump(data_TMX_train, file, indent=4)
    
with open('fine-tuning-data/TMX_dev.json', 'w',encoding="utf-8") as file:
    json.dump(data_TMX_dev, file, indent=4)

# MXLIFF processing

In [1]:
from bs4 import BeautifulSoup
import sacrebleu

In [2]:
path_MXLIFF_data_1 = "Hausa-testing/FOURCi’s CLIMATE SMART AGRICULTURE FOR RURAL WOMEN-en_gb-ha-R.mxliff"
path_MXLIFF_data_2 = "Hausa-testing/KEY TERMS AND DEFINITIONS FOR USE IN CLIMATE CHANGE EDUCATION AND SENSITIZATION IN SCHOOLS-en_gb-ha-R.mxliff"

In [3]:
def process(path_MXLIFF_data):
    data= [] 

    # Open the XML file and read its contents
    with open(path_MXLIFF_data, "r", encoding="utf8") as f:
        contents = f.read()

    # Parse the XML file using BeautifulSoup
    soup = BeautifulSoup(contents, "xml")

    # Find all trans-unit tags
    trans_units = soup.find_all("trans-unit")

    # Loop through the trans-unit tags and extract the source and target text
    for trans_unit in trans_units:
        source = trans_unit.find("source")
        target = trans_unit.find("target")
        alt_trans = trans_unit.find("alt-trans", {"origin" : "machine-trans"})
        if source is not None and target is not None:
            print(f"Source: {source.text}")
            print(f"Target: {target.text}")
            print(f"alt_trans: {alt_trans.text}\n")

            data.append({'source': source.text,
                            'target': target.text,
                            'mt': alt_trans.text})
            
    return data

In [4]:
doc_1 = process(path_MXLIFF_data_1)
doc_2 = process(path_MXLIFF_data_2)

Source: {b>FOURCi’s CLIMATE SMART AGRICULTURE FOR RURAL WOMEN<b}
Target: {b>HANYAR KYAUTATA NOMAN ZAMANANI TA FOURCi’s GA MATAN KARKARA<b}
alt_trans: {b>HANYA HANYA mai wayo ga matan karkara<b}

Source: Women are experiencing increased responsibilities and workloads, especially as they are returning to their homes after the insurgency experienced in the northeast.
Target: Mata na fuskantar ƙaruwar ɗaukar nauyi da ayyuka, musamman lokutan da suke komawa gidajensu bayan fusakantar matsalar tsaro a yankin Arewa maso Gabas.
alt_trans: Mata suna fuskantar ƙarin nauyi da nauyin aiki, musamman yayin da suke komawa gidajensu bayan tashin hankali da aka fuskanta a arewa maso gabas.

Source: {b>Concern on climate Change for the Community Initiative (FOURCi) <b}continues to find more ways to support women’s agriculture activities that will overcome the gender gap in productivity and promote gender equality for the benefits of their communities and society, while also understanding the critical ro

In [6]:
src = []
tgt = []
mt = []
for seg in doc_1:
    src.append(seg['source'])
    tgt.append(seg['target'])
    mt.append(seg['mt'])
    
for seg in doc_2:
    src.append(seg['source'])
    tgt.append(seg['target'])
    mt.append(seg['mt'])
    
print(len(tgt))
print(len(mt))

200
200


In [8]:
# Calculate BLEU
bleu = sacrebleu.corpus_bleu(mt, [tgt], tokenize='flores200')
print("BLEU:", round(bleu.score, 2))

# Calculate CHRF
chrf = sacrebleu.corpus_chrf(mt, [tgt])
print("CHRF:", round(chrf.score, 2))

# Calculate TER
metric = sacrebleu.metrics.TER()
ter = metric.corpus_score(mt, [tgt])
print("TER:", round(ter.score, 2))   

Tokenizer 'spm' has been changed to 'flores101', and may be removed in the future.


BLEU: 35.53
CHRF: 58.07
TER: 64.02


In [8]:
with open('Hausa-testing/fourci-test.hau','w') as f_hau, open('Hausa-testing/fourci-test.eng','w') as f_eng, open('Hausa-testing/fourci-test.eng.inference.hau', 'w') as f_mt:
    for s,t,i in zip(src,tgt,mt):
        f_eng.write(s+'\n')
        f_hau.write(t+'\n')
        f_mt.write(i+'\n')
    
    